This notebook explores possibilities to perform bulk actions on the SSH Open Marketplace. Start with mass rejection of ingested items (CLARIN resource families).

In [ ]:
import pandas as pd #to manage dataframes
import json #to manage json objects
import requests #to interact with the API
#import the MarketPlace Library 
from sshmarketplacelib import MPData as mpd
from sshmarketplacelib import eval as eva, helper as hel

In [ ]:
mpdata = mpd()
utils=hel.Util()
check=eva.URLCheck()

In [ ]:

bearer=mpdata.checkCredentials()
headers = {'Authorization':bearer}
status = 'ingested'
source = 'CLARIN Resource Families'
url = 'https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/item-search?d.status='+status+'&f.source='+source
r2 = requests.get(url, headers=headers).json()

items =[]
for page in range(0, r2['pages']+1):
    dataclarin = requests.get(url, headers=headers).json()
    number_items = len(dataclarin['items'])
    for x in range(0, number_items):
        items.append(dataclarin['items'][x])
    
df = pd.DataFrame(items)
df

In [ ]:
bearer=mpdata.checkCredentials()
headers = {'Authorization':bearer}
for row in df.index:
    ID = df['persistentId'][row]
    url = 'https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/datasets/'+ID
    delete_item = requests.delete(url, headers=headers)
    print(delete_item)